In [29]:
import pandas as pd
import json
import LLDA.model.labeled_lda as llda
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import jensenshannon

In [54]:
with open('dim_sample.json') as f:
    all_data = json.load(f)

def get_unique_dis(data):
    disciplines = set()
    for pro_id in data.keys():
        disciplines.update(data[pro_id]['disciplines'])
    return disciplines

In [55]:
split = 180
train_data = {k: all_data[k] for k in list(all_data.keys())[:split]}
test_data = {k: all_data[k] for k in list(all_data.keys())[split:]}

train_project_data = {}
train_pub_data = []
for pro_id in train_data.keys():
    train_project_data[pro_id] = (train_data[pro_id]['title'], train_data[pro_id]['disciplines'])
    for pub_id in train_data[pro_id]['publications'].keys():
        train_pub_data.append((pub_id, train_data[pro_id]['publications'][pub_id]['title'], train_data[pro_id]['publications'][pub_id]['disciplines']))

test_project_data = {}
test_pub_data = []
for pro_id in test_data.keys():
    test_project_data[pro_id] = (test_data[pro_id]['title'], test_data[pro_id]['disciplines'])
    for pub_id in test_data[pro_id]['publications'].keys():
        test_pub_data.append((pub_id, test_data[pro_id]['publications'][pub_id]['title'], test_data[pro_id]['publications'][pub_id]['disciplines']))

print(len(train_project_data), len(train_pub_data))
print(len(test_project_data), len(test_pub_data))
print(len(get_unique_dis(train_data)), len(get_unique_dis({pub[0]: {'disciplines': pub[2]} for pub in train_pub_data})))

180 506
45 129
170 170


In [56]:
print(train_project_data[list(train_project_data.keys())[0]])
print(train_pub_data[0][1:])

('Quantifying trophic niches to measure the resilience of marine predators', ['4101', '4104', '3103'])
('Towards unlocking the trophic roles of rarely encountered squid: Opportunistic samples of Taningia danae and a Chiroteuthis aff. veranii reveal that the Southern Ocean top predators are nutrient links connecting deep-sea and shelf-slope environments', ['3103'])


In [57]:
# # load from disk
# llda_model_project = llda.LldaModel()
# llda_model_project.load_model_from_dir("LLDA/data_models/pro_abstracts_disciplines", load_derivative_properties=False)
# print("llda_model_project", llda_model_project)
# # print("Top-5 terms of topic 'virus': ", llda_model_project.top_terms_of_topic("virus", 5, False))
# print("Doc-Topic Matrix: \n", llda_model_project.theta)
# print("Topic-Term Matrix: \n", llda_model_project.beta)

In [58]:
llda_model_project = llda.LldaModel(labeled_documents=train_project_data.values())
print(llda_model_project)
llda_model_pub = llda.LldaModel(labeled_documents=[(pub[1], pub[2]) for pub in train_pub_data])
print(llda_model_pub)


Labeled-LDA Model:
	K = 171
	M = 180
	T = 1251
	WN = 2120
	LN = 281
	alpha = 0.29239766081871343
	eta = 0.001
	perplexity = 90.39256989643876
	

Labeled-LDA Model:
	K = 171
	M = 506
	T = 2987
	WN = 6374
	LN = 704
	alpha = 0.29239766081871343
	eta = 0.001
	perplexity = 201.0754566827323
	


In [59]:
# training
llda_model_project.training(iteration=10, log=True)
# while True:
#     print("iteration %s sampling..." % (llda_model.iteration + 1))
#     llda_model.training(1)
#     print("after iteration: %s, perplexity: %s" % (llda_model.iteration, llda_model.perplexity()))
#     print("delta beta: %s" % llda_model.delta_beta)
#     if llda_model.is_convergent(method="beta", delta=0.01):
#         break

after iteration: 0, perplexity: 90.39256989643876
gibbs sample count:  2120
after iteration: 1, perplexity: 39.92193920191872
gibbs sample count:  2120
after iteration: 2, perplexity: 36.904367522287714
gibbs sample count:  2120
after iteration: 3, perplexity: 36.22049551723303
gibbs sample count:  2120
after iteration: 4, perplexity: 35.19380352978232
gibbs sample count:  2120
after iteration: 5, perplexity: 35.30325445294307
gibbs sample count:  2120
after iteration: 6, perplexity: 34.10990889632067
gibbs sample count:  2120
after iteration: 7, perplexity: 34.212801494744745
gibbs sample count:  2120
after iteration: 8, perplexity: 33.5082742852733
gibbs sample count:  2120
after iteration: 9, perplexity: 33.429986347868635
gibbs sample count:  2120


In [60]:
llda_model_pub.training(iteration=10, log=True)

after iteration: 0, perplexity: 201.0754566827323
gibbs sample count:  6374
after iteration: 1, perplexity: 113.08582653475284
gibbs sample count:  6374
after iteration: 2, perplexity: 105.78100432659338
gibbs sample count:  6374
after iteration: 3, perplexity: 103.6696121806826
gibbs sample count:  6374
after iteration: 4, perplexity: 100.81403605884597
gibbs sample count:  6374
after iteration: 5, perplexity: 99.40253019969909
gibbs sample count:  6374
after iteration: 6, perplexity: 98.51498221680718
gibbs sample count:  6374
after iteration: 7, perplexity: 97.83194282569912
gibbs sample count:  6374
after iteration: 8, perplexity: 96.60624813930383
gibbs sample count:  6374
after iteration: 9, perplexity: 95.10814504066211
gibbs sample count:  6374


In [61]:
# # update
# print("before updating: ", llda_model)
# update_labeled_documents = [("new example test example test example test example test", ["example", "test"])]
# llda_model.update(labeled_documents=update_labeled_documents)
# print("after updating: ", llda_model)

# # train again
# # llda_model.training(iteration=10, log=True)
# while True:
#     print("iteration %s sampling..." % (llda_model.iteration + 1))
#     llda_model.training(1)
#     print("after iteration: %s, perplexity: %s" % (llda_model.iteration, llda_model.perplexity()))
#     print("delta beta: %s" % llda_model.delta_beta)
#     if llda_model.is_convergent(method="beta", delta=0.01):
#         break

In [62]:
# inference
# note: the result topics may be different for difference training, because gibbs sampling is a random algorithm
document = list(test_project_data.values())[0][0]
print(document)

topics = llda_model_project.inference(document=document, iteration=30, times=10)
# print sorted topics
sorted(topics, key=lambda x: x[0])

Center of Excellence in Complex Disease Genetics-from Discovery to Precision Medicine


[('3001', 0.005414771496642827),
 ('3002', 0.005414771496642827),
 ('3003', 0.005414771496642827),
 ('3004', 0.005414771496642827),
 ('3005', 0.005414771496642827),
 ('3006', 0.012822178904050215),
 ('3007', 0.005414771496642827),
 ('3008', 0.005414771496642827),
 ('3009', 0.005414771496642827),
 ('3101', 0.005414771496642827),
 ('3102', 0.005414771496642827),
 ('3103', 0.005414771496642827),
 ('3104', 0.005414771496642827),
 ('3105', 0.005414771496642827),
 ('3106', 0.005414771496642827),
 ('3107', 0.005414771496642827),
 ('3108', 0.005414771496642827),
 ('3109', 0.005414771496642827),
 ('3201', 0.005414771496642827),
 ('3202', 0.005414771496642827),
 ('3203', 0.005414771496642827),
 ('3204', 0.005414771496642827),
 ('3205', 0.005414771496642827),
 ('3206', 0.005414771496642827),
 ('3207', 0.005414771496642827),
 ('3208', 0.005414771496642827),
 ('3209', 0.005414771496642827),
 ('3210', 0.005414771496642827),
 ('3211', 0.005414771496642827),
 ('3212', 0.007266623348494674),
 ('3213', 

In [63]:
documents = list(test_project_data.values())
project_matrix_test = np.zeros((len(documents), llda_model_project.K))

for i, (document, _) in enumerate(documents):
    topics = llda_model_project.inference(document=document, iteration=100, times=10)
    sorted_topics = sorted(topics, key=lambda x: x[0])
    for j, (topic, prob) in enumerate(sorted_topics):
        project_matrix_test[i, j] = prob

project_matrix_test

array([[0.00541477, 0.00541477, 0.00541477, ..., 0.00541477, 0.00541477,
        0.05726662],
       [0.00551694, 0.00551694, 0.00551694, ..., 0.00551694, 0.00551694,
        0.04136599],
       [0.00562303, 0.00562303, 0.00562303, ..., 0.00562303, 0.00562303,
        0.03062303],
       ...,
       [0.00541477, 0.00541477, 0.00541477, ..., 0.00541477, 0.00541477,
        0.04985922],
       [0.00551694, 0.00551694, 0.00551694, ..., 0.00551694, 0.00551694,
        0.04702637],
       [0.00562303, 0.00562303, 0.00562303, ..., 0.00562303, 0.00562303,
        0.02100765]])

In [64]:
documents = [(pub[1], pub[2]) for pub in test_pub_data]
pub_matrix_test = np.zeros((len(documents), llda_model_pub.K))

for i, (document, _) in enumerate(documents):
    topics = llda_model_pub.inference(document=document, iteration=100, times=10)
    sorted_topics = sorted(topics, key=lambda x: x[0])
    for j, (topic, prob) in enumerate(sorted_topics):
        pub_matrix_test[i, j] = prob

pub_matrix_test

array([[0.00471609, 0.00471609, 0.00471609, ..., 0.00471609, 0.00471609,
        0.05471609],
       [0.0071345 , 0.00531632, 0.00531632, ..., 0.00531632, 0.00531632,
        0.0071345 ],
       [0.00495589, 0.00495589, 0.00495589, ..., 0.00495589, 0.00495589,
        0.01851521],
       ...,
       [0.00487329, 0.00487329, 0.00487329, ..., 0.00487329, 0.00487329,
        0.02487329],
       [0.00643275, 0.0047934 , 0.0047934 , ..., 0.0047934 , 0.0047934 ,
        0.02282619],
       [0.00464123, 0.00622853, 0.00464123, ..., 0.00464123, 0.00464123,
        0.02368885]])

In [65]:
project_matrix = np.concatenate((llda_model_project.theta, project_matrix_test))
pub_matrix = np.concatenate((llda_model_pub.theta, pub_matrix_test))

distance_matrix = np.zeros((len(project_matrix), len(pub_matrix)))

for i in range(len(project_matrix)):
    for j in range(len(pub_matrix)):
        distance_matrix[i, j] = jensenshannon(project_matrix[i], pub_matrix[j])

In [68]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# create a dataframe with row indices as project ids and column indices as publication ids
# if publication is in project, then 1, else 0
# if publication date is before project start date, then 0, else 1
# ratio of intersection of organisations from project and publication to organisations from publication
# ratio of intersection of authors from project and publication to authors from publication

all_data = {**train_data, **test_data}
pub_data = []
for pro_id in all_data.keys():
    for pub_id in all_data[pro_id]['publications'].keys():
        pub_data.append((pub_id, all_data[pro_id]['publications'][pub_id]))

data_matrix = [[0.] * len(pub_data) for _ in range(len(all_data))]
for i, pro_id in enumerate(all_data.keys()):
    for j, pub_id in enumerate([pub[0] for pub in pub_data]):
        belongs = 1 if pub_id in all_data[pro_id]['publications'].keys() else 0
        distance = distance_matrix[i, j]
        data_matrix[i][j] = [belongs, distance]

df = pd.DataFrame(distance_matrix, index=all_data.keys(), columns=[pub[0] for pub in pub_data])
df.to_csv('distance_dim_title.csv')
df.head()

,10.3389/fmars.2023.1254461,10.1007/978-3-031-49133-7_8,10.1007/978-3-031-43619-2_15,10.1080/11663081.2023.2246863,10.1177/03057356231165327,10.1016/j.jconrel.2024.05.037,10.1007/s13346-024-01618-6,10.1002/smll.202303682,10.1371/journal.pone.0298895,10.1007/s00334-024-00992-y,10.1038/s41598-023-44445-4,10.1101/2023.09.15.557939,10.1016/j.jasrep.2023.104204,10.1016/j.palaeo.2023.111655,10.1007/jhep05(2024)154,10.1140/epjc/s10052-024-12711-y,10.1140/epjc/s10052-024-12416-2,10.1140/epjc/s10052-024-12415-3,10.1007/jhep02(2024)124,10.1007/jhep11(2023)103,10.1007/jhep06(2023)086,10.1093/iob/obad026,10.1007/s00359-023-01629-7,10.1101/2023.03.11.531676,10.1093/mnras/stae997,10.1093/mnras/stae616,10.1051/0004-6361/202348958,10.1093/mnras/stad3163,10.1093/mnras/stad2183,10.1101/2024.01.14.574481,10.1007/s40313-024-01095-9,10.1109/tpwrs.2023.3321700,10.1109/tits.2024.3392914,10.1007/978-981-97-0989-2_24,10.1109/tsc.2023.3332701,10.1007/978-3-031-48424-7_15,10.1007/978-3-031-48424-7_13,10.1145/3631353,10.1145/3629517,10.1109/icdcs57875.2023.00111,10.1109/icws60048.2023.00056,10.1109/icws60048.2023.00031,10.1109/icws60048.2023.00068,10.1109/tsc.2023.3307143,10.1145/3600232,10.1109/mic.2023.3267266,10.1109/tsc.2023.3241975,10.1109/tmc.2022.3230856,10.1007/978-3-031-20984-0_9,10.1109/tsc.2022.3160469,10.3389/fneur.2023.1182561,10.1038/s41598-024-57498-w,10.1016/j.jmrt.2023.11.014,10.1016/j.jmst.2023.05.020,10.1016/j.fuel.2023.130187,10.1017/9781009297349,10.1007/s11229-023-04282-4,10.3390/rel14060758,10.1002/smsc.202400096,10.1016/j.actpsy.2024.104252,10.1080/02691728.2024.2326828,10.5964/ejop.12031,10.1111/jopy.12805,10.1109/lsp.2023.3277792,10.21203/rs.3.rs-3644422/v1,10.1016/j.renene.2024.120101,10.1093/rheumatology/kead301,10.1097/mlr.0000000000002000,10.1111/tmi.13827,10.1063/5.0167167,10.1021/acsami.2c19376,10.1021/acsanm.2c05047,10.1101/2024.05.21.595254,10.1038/s41467-024-47964-4,10.1093/jb/mvae021,10.1016/j.neuron.2024.01.014,10.1016/j.isci.2023.107385,10.1016/j.peptides.2023.171051,10.3389/fphar.2023.1138666,10.3389/fphar.2023.1098976,10.1186/s43058-023-00521-4,10.1177/26334895231199467,10.1002/adom.202303199,10.1002/adom.202301232,10.1016/j.energy.2024.130322,10.1126/sciimmunol.abo4365,10.1515/sem-2023-0178,10.1364/boe.506664,10.1364/boe.494585,10.3847/1538-4357/ac4601,10.1371/journal.pone.0284416,10.1080/07350015.2023.2271039,10.1093/mnras/stae385,10.1093/mnras/stad2760,10.1002/ange.202320045,10.1002/anie.202320045,10.1016/j.chembiol.2024.02.012,10.1101/2023.11.29.569293,10.1038/s41467-023-41197-7,10.35772/ghm.2023.01065,10.3390/v15040896,10.1111/dar.13836,10.1063/5.0153862,10.1109/imoc57131.2023.10379741,10.1109/imoc57131.2023.10379729,10.1109/access.2023.3339563,10.1109/ojpel.2023.3339014,10.1109/csci58124.2022.00203,10.1016/j.ssci.2022.105967,10.1007/978-3-031-06015-1_8,10.1101/2023.01.19.524726,10.1007/s00382-023-07058-1,10.1002/oa.3284,10.1136/bmjopen-2023-080603,10.1101/2023.08.02.551649,10.1099/mgen.0.000965,10.2140/ant.2022.16.2005,10.1103/physrevlett.132.103402,10.21468/scipostphys.15.2.068,10.1080/02606755.2023.2213550,10.3390/rs15204928,10.1063/5.0203603,10.1103/physrevapplied.21.054027,10.1109/icecom58258.2023.10367924,10.1109/icecom58258.2023.10367934,10.1109/icecom58258.2023.10367938,10.1109/icecom58258.2023.10367923,10.1109/metamaterials58257.2023.10289546,10.23919/ursigass57860.2023.10265525,10.1109/elmar59410.2023.10253914,10.1002/pen.26771,10.1016/j.canep.2023.102451,10.3390/biomedicines11051344,10.3390/pharmaceutics15010181,10.1037/fam0001189,10.3390/su15129371,10.1038/s44221-024-00208-7,10.1093/evolut/qpae036,10.1093/evolut/qpae024,10.3390/micro4010008,10.3390/molecules28196822,10.1007/s11673-023-10285-0,10.1021/acsnano.3c09776,10.1101/2023.09.08.556607,10.1016/j.snb.2023.133733,10.1016/j.ocecoaman.2023.106961,10.1016/j.tre.2023.103169,10.1016/j.elerap.2023.101336,10.1590/0001-3765202320230351,10.1590/s1678-3921.pab2023.v58.03246,10.1038/s41598-024-56540-1,10.1364/ol.494369,10.1017/s0963180124000112,10.1136/jm

In [69]:
data_matrix = np.array(data_matrix)

gb_raw_shape = data_matrix.shape
print(gb_raw_shape)

gb_data = []
for i in range(gb_raw_shape[1]):
    gb_data.extend(data_matrix[:, i, :])
gb_data = np.array(gb_data)
print(gb_data.shape)

project_numbers = np.array(list(range(1, gb_raw_shape[0] + 1)) * gb_raw_shape[1])

project_numbers = np.array(list(range(1, gb_raw_shape[0] + 1)) * gb_raw_shape[1])

# get the projects for lowest predictions for each publication with distance matrix
for num_top in range(10, 0, -1):
    project_numbers_pred = []
    for i in range(len(train_pub_data), distance_matrix.shape[1]):
        project_numbers_pred.append(list(np.argsort(distance_matrix[:, i] + 1))[:num_top])

    project_lists = []
    for i in range(0, gb_data.shape[0], gb_raw_shape[0]):
        pub_belongs = gb_data[i:i + gb_raw_shape[0], 0] * project_numbers[i:i + gb_raw_shape[0]]
        project = sorted(list(pub_belongs[pub_belongs != 0].astype(int)), reverse=True)
        project_lists.append(project)

    project_lists_test = project_lists[len(train_pub_data):]

    # calculate accuracy
    # use y_test as mask for project_numbers_test
    accuracy_list = []
    for i, project_list in enumerate(list(project_lists_test)):
        ratio = len(set(project_numbers_pred[i]) & set(project_list)) / len(set(project_list))
        accuracy_list.append(ratio)

    accuracy = sum(accuracy_list) / len(accuracy_list)
    print(accuracy)

(225, 635, 2)
(142875, 2)
0.16279069767441862
0.14728682170542637
0.13178294573643412
0.11627906976744186
0.09302325581395349
0.07751937984496124
0.06201550387596899
0.05426356589147287
0.03875968992248062
0.023255813953488372
